# Chapter 12: Type Aliases

---

## 12.1 Creating Type Aliases

Type aliases provide a way to create a new name for a type. Unlike interfaces which define a new type, type aliases simply create a reference to an existing type, making your code more readable, maintainable, and self-documenting.

### 12.1.1 Basic Type Alias Syntax

**The `type` Keyword:**

```typescript
// Creating a type alias for a primitive
type UserId = string;
type Age = number;
type IsActive = boolean;

// Using the aliases
const userId: UserId = "user-123";  // Still just a string, but semantically named
const age: Age = 30;
const active: IsActive = true;

// Type aliases for object types
type User = {
  id: UserId;        // Can use other type aliases
  name: string;
  age: Age;
  isActive: IsActive;
};

const user: User = {
  id: "user-456",
  name: "John",
  age: 30,
  isActive: true
};

// Type aliases for union types
type Status = "pending" | "active" | "inactive";
type StringOrNumber = string | number;
type Nullable<T> = T | null | undefined;
```

**Type Alias Syntax Structure:**

```
┌─────────────────────────────────────────────────────────────────────┐
│                    Type Alias Syntax                                 │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   type AliasName = Type;                                            │
│                                                                     │
│   Examples:                                                         │
│   ┌──────────────────────────────────────────────────────────────┐ │
│   │ // Primitive aliases                                          │ │
│   │ type ID = string;                                             │ │
│   │ type Quantity = number;                                       │ │
│   │                                                                │ │
│   │ // Object type alias                                          │ │
│   │ type Point = {                                                │ │
│   │   x: number;                                                  │ │
│   │   y: number;                                                  │ │
│   │ };                                                            │ │
│   │                                                                │ │
│   │ // Union type alias                                           │ │
│   │ type Status = "active" | "inactive";                          │ │
│   │                                                                │ │
│   │ // Function type alias                                        │ │
│   │ type Callback = (data: string) => void;                       │ │
│   │                                                                │ │
│   │ // Generic type alias                                         │ │
│   │ type Container<T> = { value: T };                             │ │
│   └──────────────────────────────────────────────────────────────┘ │
│                                                                     │
│   Key Characteristics:                                              │
│   • Does NOT create a new type, just a name for existing type      │
│   • Can alias any type: primitive, object, union, tuple, etc.    │
│   • Cannot be reopened/merged (unlike interfaces)                   │
│   • Can use generics                                                │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### 12.1.2 Aliasing Primitive Types

**Semantic Type Aliases:**

```typescript
// Domain-specific type aliases make code self-documenting
type Email = string;
type PhoneNumber = string;
type URL = string;
type UUID = string;
type Timestamp = number; // Unix timestamp
type Percentage = number; // 0-100
type Money = number; // In cents to avoid floating point issues

// Using semantic types
function sendEmail(to: Email, subject: string, body: string): void {
  console.log(`Sending to ${to}: ${subject}`);
}

function formatCurrency(amount: Money): string {
  return `$${(amount / 100).toFixed(2)}`;
}

function calculateDiscount(price: Money, discount: Percentage): Money {
  return price * (1 - discount / 100);
}

// These are all strings at runtime, but TypeScript treats them distinctly
const email: Email = "user@example.com";
const phone: PhoneNumber = "555-0123";

// TypeScript won't prevent assignment between same underlying types
// But it helps with documentation and intent
const someString: string = email; // OK - Email is just a string
// const email2: Email = phone; // Technically allowed, but semantically wrong
```

**Branded Types (Opaque Types):**

```typescript
// Creating distinct types that can't be accidentally mixed
type UserId = string & { __brand: "UserId" };
type OrderId = string & { __brand: "OrderId" };
type ProductId = string & { __brand: "ProductId" };

// Factory functions that create branded types
function createUserId(id: string): UserId {
  return id as UserId;
}

function createOrderId(id: string): OrderId {
  return id as OrderId;
}

const userId = createUserId("user-123");
const orderId = createOrderId("order-456");

function getUser(id: UserId): void {
  console.log(`Fetching user ${id}`);
}

function getOrder(id: OrderId): void {
  console.log(`Fetching order ${id}`);
}

getUser(userId);      // ✅ OK
getOrder(orderId);    // ✅ OK
// getUser(orderId);  // ❌ Error: OrderId not assignable to UserId
// getOrder(userId);  // ❌ Error: UserId not assignable to OrderId

// At runtime, they're just strings
console.log(typeof userId); // "string"
```

### 12.1.3 Aliasing Object Types

**Object Type Aliases:**

```typescript
// Complex object type
type Address = {
  street: string;
  city: string;
  state: string;
  zipCode: string;
  country: string;
};

type Coordinates = {
  latitude: number;
  longitude: number;
};

// Combining type aliases
type Location = {
  address: Address;
  coordinates: Coordinates;
};

// Usage
const officeLocation: Location = {
  address: {
    street: "123 Main St",
    city: "San Francisco",
    state: "CA",
    zipCode: "94105",
    country: "USA"
  },
  coordinates: {
    latitude: 37.7749,
    longitude: -122.4194
  }
};

// Optional and readonly modifiers
type ImmutableConfig = {
  readonly apiUrl: string;
  readonly apiKey: string;
  timeout?: number;
  retries?: number;
};

const config: ImmutableConfig = {
  apiUrl: "https://api.example.com",
  apiKey: "secret123"
};

// config.apiUrl = "new"; // ❌ Error: readonly
config.timeout = 10000;     // ✅ OK - optional and mutable
```

### 12.1.4 Aliasing Union Types

**Union Type Aliases:**

```typescript
// Status unions
type HttpStatus = 200 | 201 | 204 | 400 | 401 | 403 | 404 | 500;
type RequestState = "idle" | "loading" | "success" | "error";

// Complex unions
type ApiResponse<T> = 
  | { status: "success"; data: T }
  | { status: "error"; error: string; code: number }
  | { status: "loading" };

// Result type pattern
type Result<T, E = Error> = 
  | { ok: true; value: T }
  | { ok: false; error: E };

// Usage with Result type
function parseJSON<T>(json: string): Result<T, SyntaxError> {
  try {
    const data = JSON.parse(json) as T;
    return { ok: true, value: data };
  } catch (e) {
    return { ok: false, error: e as SyntaxError };
  }
}

const result = parseJSON<{ name: string }>('{"name": "John"}');

if (result.ok) {
  console.log(result.value.name); // TypeScript knows value exists
} else {
  console.error(result.error.message); // TypeScript knows error exists
}
```

---

## 12.2 Type Aliases vs Interfaces

Understanding the differences between type aliases and interfaces is crucial for writing idiomatic TypeScript. While they can often be used interchangeably for object types, they have distinct characteristics and use cases.

### 12.2.1 Key Differences

**Declaration Merging:**

```typescript
// Interfaces support declaration merging
interface User {
  name: string;
}

interface User {
  age: number;
}

// Result: User has both name and age
const user: User = {
  name: "John",
  age: 30
};

// Type aliases do NOT support merging
type Person = {
  name: string;
};

// type Person = {  // ❌ Error: Duplicate identifier 'Person'
//   age: number;
// };

// To extend a type alias, use intersection
type PersonWithAge = Person & { age: number };
```

**Extending vs Intersection:**

```typescript
// Interface extends interface
interface Animal {
  name: string;
}

interface Dog extends Animal {
  breed: string;
}

// Type alias extends using intersection
type Animal2 = {
  name: string;
};

type Dog2 = Animal2 & {
  breed: string;
};

// Both result in same structure
const dog: Dog = { name: "Rex", breed: "German Shepherd" };
const dog2: Dog2 = { name: "Rex", breed: "German Shepherd" };

// Interface can extend type alias
interface Cat extends Animal2 {
  color: string;
}

// Type alias can intersect with interface
type Bird = Animal & {
  wingSpan: number;
};
```

**Other Differences:**

```typescript
// Interfaces can only describe object shapes (and functions via call signatures)
interface Point {
  x: number;
  y: number;
}

// Type aliases can describe any type
type Primitive = string | number | boolean;
type Tuple = [number, number];
type Callback = (data: string) => void;

// Type aliases can use utility types
type PartialUser = Partial<{ name: string; age: number }>;
type ReadonlyUser = Readonly<{ name: string; age: number }>;

// Interfaces cannot be used with utility types directly
// interface PartialInterface = Partial<...>; // ❌ Error

// Error messages differ
interface LongInterface {
  a: string;
  b: number;
  c: boolean;
  d: string[];
}

type LongType = {
  a: string;
  b: number;
  c: boolean;
  d: string[];
};

// Interface error: Property 'e' is missing in type '...' but required in type 'LongInterface'
// Type alias error: Type '...' is not assignable to type '{ a: string; b: number; c: boolean; d: string[]; }'
// (Type aliases often show the full expanded type in errors)
```

### 12.2.2 When to Use Type Aliases

**Use Type Aliases When:**

```typescript
// 1. You need to alias a primitive
type UserId = string;
type Money = number;

// 2. You need a union type
type Status = "active" | "inactive" | "pending";
type Result<T> = Success<T> | Failure;

// 3. You need a tuple type
type Point = [number, number];
type NameAndAge = [string, number];

// 4. You need to use mapped types or conditional types
type Nullable<T> = T | null;
type NonNullable<T> = T extends null | undefined ? never : T;

// 5. You need function types
type Reducer<S, A> = (state: S, action: A) => S;
type Middleware = (next: () => void) => () => void;

// 6. You need recursive types (interfaces can be recursive too, but types are more flexible)
type JSONValue = 
  | string 
  | number 
  | boolean 
  | null 
  | JSONValue[] 
  | { [key: string]: JSONValue };

// 7. You don't want declaration merging (sealed/final type)
type Config = {
  apiUrl: string;
  timeout: number;
};
// Cannot be extended or merged elsewhere - ensures stability
```

### 12.2.3 When to Use Interfaces

**Use Interfaces When:**

```typescript
// 1. You're defining the shape of an object that will be implemented by classes
interface Drawable {
  draw(): void;
  getArea(): number;
}

class Circle implements Drawable {
  draw(): void { console.log("Drawing circle"); }
  getArea(): number { return Math.PI; }
}

// 2. You need declaration merging (extending third-party types)
// In a .d.ts file or module augmentation:
interface Window {
  myApp: {
    version: string;
  };
}

// Now window.myApp is available everywhere

// 3. You're creating a public API that might need extension
interface Plugin {
  name: string;
  initialize(): void;
}

// Consumers can extend Plugin interface to add methods

// 4. You want better error messages for object types
// Interfaces often provide clearer error messages than type aliases

// 5. You prefer the OOP style syntax
interface Animal {
  name: string;
  eat(): void;
}

// vs
type Animal2 = {
  name: string;
  eat(): void;
};
```

**Decision Matrix:**

```
┌─────────────────────────────────────────────────────────────────────┐
│              Type Alias vs Interface Decision Matrix                   │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   Use Case                    │ Type Alias │ Interface                │
│  ─────────────────────────────┼────────────┼────────────────────────  │
│   Object shape              │ ✅         │ ✅                       │
│   Class implementation        │ ❌         │ ✅                       │
│   Declaration merging         │ ❌         │ ✅                       │
│   Union types               │ ✅         │ ❌                       │
│   Primitive aliases         │ ✅         │ ❌                       │
│   Tuple types               │ ✅         │ ❌                       │
│   Function types            │ ✅         │ ⚠️ (via call signature)  │
│   Mapped/Conditional types    │ ✅         │ ❌                       │
│   Recursive types           │ ✅         │ ✅                       │
│   Extending (inheritance)   │ &          │ extends                  │
│                                                                     │
│   General Rule:                                                     │
│   • Use interface for object shapes that classes implement          │
│   • Use type alias for everything else                              │
│   • Be consistent within a project                                   │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

---

## 12.3 Advanced Type Aliases

### 12.3.1 Recursive Type Aliases

Type aliases can reference themselves, which is useful for defining tree structures, linked lists, or recursive data formats like JSON.

**Recursive Type Patterns:**

```typescript
// JSON type (recursive)
type JSONValue =
  | string
  | number
  | boolean
  | null
  | JSONValue[]           // Array of JSON values
  | { [key: string]: JSONValue }; // Object with JSON values

const validJson: JSONValue = {
  name: "John",
  age: 30,
  hobbies: ["reading", "coding"],
  address: {
    city: "NYC",
    coordinates: [40.7128, -74.0060]
  },
  metadata: null
};

// Tree structure
type TreeNode<T> = {
  value: T;
  children: TreeNode<T>[];
};

const tree: TreeNode<string> = {
  value: "root",
  children: [
    {
      value: "child1",
      children: [
        { value: "grandchild1", children: [] }
      ]
    },
    {
      value: "child2",
      children: []
    }
  ]
};

// Linked list
type LinkedList<T> = {
  head: Node<T> | null;
};

type Node<T> = {
  value: T;
  next: Node<T> | null;
};

// File system structure
type FileSystemItem =
  | {
      type: "file";
      name: string;
      size: number;
      content: string;
    }
  | {
      type: "directory";
      name: string;
      children: FileSystemItem[];
    };

const root: FileSystemItem = {
  type: "directory",
  name: "root",
  children: [
    {
      type: "file",
      name: "readme.txt",
      size: 1024,
      content: "Hello World"
    },
    {
      type: "directory",
      name: "src",
      children: [
        {
          type: "file",
          name: "index.ts",
          size: 2048,
          content: "console.log('hi')"
        }
      ]
    }
  ]
};
```

### 12.3.2 Type Aliases with Generics

Generics make type aliases flexible and reusable across different types.

**Generic Type Aliases:**

```typescript
// Container type
type Container<T> = {
  value: T;
  getValue(): T;
  setValue(newValue: T): void;
};

const stringContainer: Container<string> = {
  value: "hello",
  getValue() { return this.value; },
  setValue(newValue) { this.value = newValue; }
};

// API Response type
type ApiResponse<T, E = Error> =
  | { status: "success"; data: T }
  | { status: "error"; error: E }
  | { status: "loading" };

type User = { id: number; name: string };
type UserResponse = ApiResponse<User>;

const successResponse: UserResponse = {
  status: "success",
  data: { id: 1, name: "John" }
};

// Higher-order type
type AsyncFunction<T, R> = (input: T) => Promise<R>;
type SyncFunction<T, R> = (input: T) => R;

type MaybeAsync<T, R> = AsyncFunction<T, R> | SyncFunction<T, R>;

// Constrained generics
type NonEmptyArray<T> = [T, ...T[]];
const numbers: NonEmptyArray<number> = [1]; // Must have at least one element
// const empty: NonEmptyArray<number> = []; // ❌ Error

// Key-value mapping with constraints
type Dictionary<K extends string | number, V> = {
  [key in K]: V;
};

type StringDict<V> = Dictionary<string, V>;
const dict: StringDict<number> = {
  a: 1,
  b: 2
};
```

### 12.3.3 Mapped Types with Type Aliases

Type aliases work excellently with mapped types to create transformed versions of existing types.

**Mapped Type Aliases:**

```typescript
// Make all properties optional
type Partial<T> = {
  [K in keyof T]?: T[K];
};

// Make all properties required
type Required<T> = {
  [K in keyof T]-?: T[K]; // -? removes optionality
};

// Make all properties readonly
type Readonly<T> = {
  readonly [K in keyof T]: T[K];
};

// Remove readonly
type Mutable<T> = {
  -readonly [K in keyof T]: T[K];
};

// Pick specific keys
type Pick<T, K extends keyof T> = {
  [P in K]: T[P];
};

// Omit specific keys
type Omit<T, K extends keyof T> = Pick<T, Exclude<keyof T, K>>;

// Usage
interface User {
  id: number;
  name: string;
  email: string;
  password: string;
}

type PublicUser = Omit<User, "password">;
// { id: number; name: string; email: string }

type UserUpdate = Partial<Omit<User, "id">>;
// { name?: string; email?: string; password?: string }

// Custom mapped type
type Nullable<T> = {
  [K in keyof T]: T[K] | null;
};

type NullableUser = Nullable<User>;
// { id: number | null; name: string | null; ... }
```

---

## 12.4 Template Literal Types (Introduction)

While covered in depth in Chapter 20, here's a brief introduction to template literal types using type aliases.

**Basic Template Literal Types:**

```typescript
// Template literal types create string literal unions
type EventName<T extends string> = `on${Capitalize<T>}`;

type MouseEvents = EventName<"click" | "hover" | "leave">;
// "onClick" | "onHover" | "onLeave"

// CSS unit types
type Unit = "px" | "em" | "rem" | "%";
type CSSValue = `${number}${Unit}`;

const width1: CSSValue = "100px";    // ✅
const width2: CSSValue = "1.5rem";  // ✅
// const width3: CSSValue = "abc";   // ❌

// Route paths
type Route = `/api/${"users" | "posts" | "comments"}/${number}`;

const route1: Route = "/api/users/123";     // ✅
const route2: Route = "/api/posts/456";     // ✅
// const route3: Route = "/api/products/123"; // ❌
```

---

## 12.5 Naming Conventions for Type Aliases

Consistent naming conventions improve code readability and maintainability.

**Naming Best Practices:**

```typescript
// 1. Use PascalCase for type aliases
type UserId = string;
type HttpResponse<T> = { ... };

// 2. Use descriptive names that convey meaning
// Bad
type X = string;
type D = { ... };

// Good
type EmailAddress = string;
type UserDocument = { ... };

// 3. Suffix for specific patterns
type UserDTO = { ... };        // Data Transfer Object
type UserEntity = { ... };      // Database entity
type UserProps = { ... };       // Component props
type UserConfig = { ... };      // Configuration
type UserState = { ... };       // State management
type UserParams = { ... };      // URL/function parameters

// 4. Prefix for boolean types (optional, but common)
type IsActive = boolean;
type HasPermission = boolean;
type CanEdit = boolean;

// 5. Generic type parameter naming
type Container<T> = { ... };           // Single letter for simple cases
type Container<TItem> = { ... };       // Descriptive for clarity
type Container<TElement, TIndex> = { ... }; // Multiple generics

// 6. Avoid naming conflicts with interfaces
// If you have an interface User, don't create type User
// Instead, create type UserData or UserAttributes

// 7. Union type naming
type Status = "active" | "inactive";  // Generic noun
type UserStatus = "online" | "offline"; // Prefixed noun
type LoadingState = "idle" | "loading" | "loaded"; // Descriptive

// 8. Function type naming
// Verb + noun pattern
type ValidateInput = (input: string) => boolean;
type TransformData = <T, R>(data: T) => R;
type HandleClick = (event: MouseEvent) => void;
```

---

## 12.6 Chapter Summary and Exercises

### Chapter Summary

In this chapter, we explored type aliases comprehensively:

**Key Takeaways:**

1. **Type Alias Basics**:
   - Created with `type Name = Type`
   - Creates a reference, not a new type
   - Can alias any type: primitives, objects, unions, tuples, functions

2. **vs Interfaces**:
   - Type aliases: Cannot be merged, work with any type, use `&` for extension
   - Interfaces: Can be merged, work with objects only, use `extends` for extension
   - Use interfaces for class contracts, type aliases for everything else

3. **Advanced Patterns**:
   - Recursive type aliases for tree structures
   - Generic type aliases for reusable containers
   - Mapped types for transformations
   - Branded types for type safety

4. **Naming Conventions**:
   - PascalCase for type names
   - Descriptive, semantic naming
   - Consistent suffixes (DTO, Props, Config, etc.)

### Practical Exercises

**Exercise 1: Basic Type Aliases**

Create type aliases for a domain:

```typescript
// 1. Create type aliases for:
//    - UserId (string), Email (string), Timestamp (number)
//    - Money (number in cents), Percentage (number 0-100)
//    - Status: "active" | "inactive" | "suspended"

// 2. Create an Address type with street, city, state, zip, country

// 3. Create a User type using the aliases above plus:
//    - id: UserId
//    - email: Email
//    - createdAt: Timestamp
//    - balance: Money
//    - discountRate: Percentage
//    - status: Status
//    - address: Address

// 4. Create a function that accepts a User and returns a formatted string
//    showing their balance as dollars (divide Money by 100)
```

**Exercise 2: Union Type Aliases**

Build a notification system:

```typescript
// 1. Create a Notification type that is a union of:
//    - EmailNotification: { channel: "email"; to: string; subject: string; body: string }
//    - SMSNotification: { channel: "sms"; phone: string; message: string }
//    - PushNotification: { channel: "push"; deviceToken: string; title: string; body: string }

// 2. Create a sendNotification function that:
//    - Accepts a Notification
//    - Uses type narrowing to handle each channel differently
//    - Returns a DeliveryResult type (union of success/failure)

// 3. Create a NotificationSummary type that extracts just the common fields
//    (hint: use Pick or manual extraction)
```

**Exercise 3: Recursive Types**

Model a file system:

```typescript
// 1. Create a FileSystemNode type that represents:
//    - File: { type: "file"; name: string; size: number; content: string }
//    - Directory: { type: "directory"; name: string; children: FileSystemNode[] }

// 2. Write a function calculateSize(node: FileSystemNode): number that:
//    - Returns file size for files
//    - Recursively calculates total size for directories

// 3. Write a function findFile(node: FileSystemNode, name: string): File | null
//    that searches recursively for a file by name

// 4. Create a sample file system tree with nested directories and test your functions
```

**Exercise 4: Type Aliases vs Interfaces**

Compare and contrast:

```typescript
// 1. Define a User using interface:
//    interface IUser { name: string; age: number; }

// 2. Define the same using type alias:
//    type TUser = { name: string; age: number; }

// 3. Demonstrate:
//    - How to extend each (create AdminUser with extra permissions field)
//    - Attempt declaration merging with both (show which works)
//    - Use with generic utility types (Partial, Pick, etc.)

// 4. Write a short explanation of when you'd choose each for a new project
```

**Exercise 5: Generic Type Aliases**

Create reusable utility types:

```typescript
// 1. Create a type AsyncResult<T> that represents:
//    - Loading state
//    - Success with data T
//    - Error with message and code

// 2. Create a type Store<T> that represents a simple data store with:
//    - data: T[]
//    - find(id: string): T | undefined
//    - add(item: T): void
//    - remove(id: string): boolean

// 3. Create a type EventHandler<TEvent> for typed event handling

// 4. Create a type Validator<T> that represents a function (value: T) => boolean | string
//    (returns true if valid, or error message if invalid)

// 5. Use these types to implement a UserStore with validation
```

### Additional Resources

- **TypeScript Handbook - Type Aliases**: https://www.typescriptlang.org/docs/handbook/2/everyday-types.html#type-aliases
- **TypeScript Handbook - Interfaces vs Type Aliases**: https://www.typescriptlang.org/docs/handbook/2/everyday-types.html#differences-between-type-aliases-and-interfaces
- **Recursive Type Aliases**: https://www.typescriptlang.org/docs/handbook/release-notes/typescript-3-7.html#more-recursive-type-aliases
- **Effective TypeScript - Item 13**: https://effectivetypescript.com/2020/05/12/unsoundness/

---

## Coming Up Next: Chapter 13 - Type Guards and Type Predicates

In the next chapter, we will explore:

- Understanding type guards and their importance
- Built-in type guards: `typeof`, `instanceof`, `in`
- User-defined type guards using the `is` operator
- Assertion functions with the `asserts` keyword
- Creating custom type guard functions
- Type guard best practices and patterns

Type guards are essential tools for narrowing union types and working safely with dynamic data, enabling you to write type-safe code that handles multiple possible shapes of data at runtime.

---